<a href="https://colab.research.google.com/github/atstuyuki/ultralytics/blob/main/yolov8_detect_on_colab_shoulderXray_20250925.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 参考記事
https://blog.roboflow.com/train-a-custom-yolov8-pose-estimation-model/


## 必要なライブラリのインストール

In [1]:
%pip install ultralytics
from ultralytics import YOLO

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.2 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


## 学習済みモデル(best.pt)をuploadしてmodelという変数に格納


In [2]:
from google.colab import files
from ultralytics import YOLO

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  # Assume the uploaded file is the best.pt and load it
  model = YOLO(fn)

print("Model loaded successfully.")

Saving best.pt to best.pt
User uploaded file "best.pt" with length 6423953 bytes
Model loaded successfully.


検出させたい動画をuploadしvideo_sourece_pathという変数に格納

In [9]:
from google.colab import files
import os

uploaded_video = files.upload()

# Assuming only one video file is uploaded
if uploaded_video:
    video_filename = list(uploaded_video.keys())[0]
    video_source_path = os.path.join('/content/', video_filename)
    print(f'Uploaded video "{video_filename}" and stored path in video_source_path.')
else:
    video_source_path = None
    print("No video file uploaded.")

Saving IS_shoulder.mp4 to IS_shoulder.mp4
Uploaded video "IS_shoulder.mp4" and stored path in video_source_path.


##検出実行！ 検出後の動画は自動的にダウンロードされる

In [10]:
# Perform inference on a video
# Use the defined video_source_path
results = model.predict(source=video_source_path, save=True)

# Find the saved video file and download it
import glob
import os
from google.colab import files

# The saved video will be in a 'runs' directory, typically in a 'predict' folder
# We look for a file with the same base name as the source video in the 'runs' directory structure
video_filename_base = os.path.splitext(os.path.basename(video_source_path))[0]
search_pattern = f'runs/**/{video_filename_base}.avi' # YOLO saves as .avi by default for videos

saved_video_paths = glob.glob(search_pattern, recursive=True)

if saved_video_paths:
    # Assuming the first found file is the one we want
    saved_video_path = saved_video_paths[0]
    print(f"Downloading saved video: {saved_video_path}")
    files.download(saved_video_path)
else:
    print("Could not find the saved video file.")


# You can access the results of the inference
# For example, to see the results for the first frame:
#print(results[0])


WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/160) /content/IS_shoulder.mp4: 640x640 1 clavicle, 1 humeral, 1 scapula, 8.9ms
video 1/1 (frame 2/160) /content/IS_shoulder.mp4: 640x640 1 clavicle, 1 humeral, 1 scapula, 9.7ms
video 1/1 (frame 3/160) /content/IS_shoulder.mp4: 640x640 1 clavicle, 1 humeral, 1 scapula, 8.8ms
video 1/1 (frame 4/160) /content/IS_shoulder.mp4: 640x640 1 clavicle, 1 humeral, 1 scapula, 9.4ms
video 1/1 (frame 5/160) /content/IS_shoulder.mp4: 640x640 1 clav

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 検出座標から推定した角度をCSVに保存してダウンロード

In [13]:
import pandas as pd
import os
from google.colab import files
import numpy as np
import cv2 # Import OpenCV for drawing
from google.colab.patches import cv2_imshow # Import cv2_imshow for displaying images in Colab

# Dictionary to store angles by frame and class
angle_data_dict = {}

# Open the video file
cap = cv2.VideoCapture(video_source_path)

# Check if video opened successfully
if not cap.isOpened():
    print(f"Error: Could not open video file {video_source_path}")
else:
    frame_num = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break # Break the loop if no more frames

        # Process the current frame
        results_frame = model.predict(source=frame, save=False, verbose=False) # predict on the current frame

        if results_frame:
            # Assuming only one result object per frame when processing frame by frame
            result = results_frame[0]
            boxes = result.boxes
            keypoints = result.keypoints
            names = result.names

            if frame_num not in angle_data_dict:
                angle_data_dict[frame_num] = {'frame_num': frame_num}

            if boxes:
                for i, box in enumerate(boxes):
                    class_id = int(box.cls)
                    class_name = names[class_id]

                    if keypoints and keypoints.xy.shape[1] > 0:
                        # Draw keypoints and add text
                        for kp_idx, (x, y) in enumerate(keypoints.xy[i].cpu().numpy()):
                             # Check if keypoint is visible (not -1, -1)
                            if x > 0 and y > 0:
                                cv2.circle(frame, (int(x), int(y)), 5, (0, 255, 0), -1) # Draw a green circle
                                # Add text for keypoint name and index
                                text = f"{class_name} KP{kp_idx}"
                                cv2.putText(frame, text, (int(x) + 10, int(y)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA) # Draw white text


                    # Calculate and store angle if enough keypoints are available
                    if keypoints.xy.shape[1] >= 2:
                        kp1 = keypoints.xy[i][0].cpu().numpy()
                        kp2 = keypoints.xy[i][1].cpu().numpy()

                        # Ensure kp1 and kp2 are valid keypoints (not -1, -1)
                        if kp1[0] > 0 and kp1[1] > 0 and kp2[0] > 0 and kp2[1] > 0:
                            vector = kp2 - kp1
                            vertical_vector = np.array([0, 1])
                            dot_product = np.dot(vector, vertical_vector)
                            magnitude_vector = np.linalg.norm(vector)
                            magnitude_vertical = np.linalg.norm(vertical_vector)

                            # Avoid division by zero
                            if magnitude_vector > 0 and magnitude_vertical > 0:
                                cosine_angle = dot_product / (magnitude_vector * magnitude_vertical)
                                angle_radians = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
                                angle_degrees = np.degrees(angle_radians)

                                # Store angle with class name in the dictionary
                                angle_data_dict[frame_num][f'{class_name}_angle'] = angle_degrees
                            else:
                                # Handle cases where the vector magnitude is zero (keypoints are the same)
                                angle_data_dict[frame_num][f'{class_name}_angle'] = np.nan # Or some other indicator


            # Display the image with keypoints for the current frame (optional, can be slow for videos)
            # if frame_num % 30 == 0: # Display every 30th frame as an example
            #      cv2_imshow(frame)

        frame_num += 1

    # Release the video capture object
    cap.release()
    # cv2.destroyAllWindows() # Not needed in Colab

# Create a DataFrame from the dictionary values
# Check if angle_data_dict is not empty before creating DataFrame
if angle_data_dict:
    angle_df = pd.DataFrame(list(angle_data_dict.values()))

    # Reorder columns to have frame_num first and then the angle columns
    # Get the angle columns dynamically based on the detected classes
    # Need to get names from the model or the first result if available
    if 'names' in locals(): # Check if names was populated during processing
        angle_cols = [f'{name}_angle' for name in names.values() if f'{name}_angle' in angle_df.columns]
        cols = ['frame_num'] + angle_cols
        # Ensure all required columns are present before reindexing
        cols = [col for col in cols if col in angle_df.columns]
        angle_df = angle_df[cols]

    # Convert clavicle_angle by subtracting from 90
    if 'clavicle_angle' in angle_df.columns:
        angle_df['clavicle_angle'] = 90 - angle_df['clavicle_angle']


    # Display the DataFrame
    display(angle_df)

    # Get the video filename from the source path stored in the variable
    video_filename = os.path.basename(video_source_path)
    csv_filename = os.path.splitext(video_filename)[0] + '_angles.csv'

    # Save the DataFrame to a CSV file
    angle_df.to_csv(csv_filename, index=False)

    # Make the CSV file downloadable
    files.download(csv_filename)

    print(f"DataFrame created and saved as '{csv_filename}'")
else:
    print("No angle data was generated. Check the detection results and keypoint validity.")

,frame_num,clavicle_angle,humeral_angle,scapula_angle
0,0,9.132876,1.546330,5.406215
1,1,9.221215,1.865610,5.570155
2,2,9.144481,2.271621,4.500821
3,3,9.167590,1.938636,4.502543
4,4,9.130386,1.575077,4.876513
...,...,...,...,...
155,155,6.920125,3.084488,6.906753
156,156,6.898214,2.994344,5.649407
157,157,7.510583,4.307435,5.747978
158,158,7.057876,3.139593,5.470401


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

DataFrame created and saved as 'IS_shoulder_angles.csv'


from matplotlib import pyplot as plt
angle_df['frame_num'].plot(kind='hist', bins=20, title='frame_num')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['frame_num']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'frame_num'}, axis=1)
              .sort_values('frame_num', ascending=True))
  xs = counted['frame_num']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = angle_df.sort_values('frame_num', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('frame_num')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
angle_df['frame_num'].plot(kind='line', figsize=(8, 4), title='frame_num')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
angle_df['clavicle_angle'].plot(kind='line', figsize=(8, 4), title='clavicle_angle')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
angle_df['humeral_angle'].plot(kind='line', figsize=(8, 4), title='humeral_angle')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
angle_df['scapula_angle'].plot(kind='line', figsize=(8, 4), title='scapula_angle')
plt.gca().spines[['top', 'right']].set_visible(False)

## 他の動画を解析したい場合は動画のupload以降を繰り返す